<a href="https://colab.research.google.com/github/Strojove-uceni/23206-final-pose-estimation-for-swing-improvement/blob/main/Yolov7_Pose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/WongKinYiu/yolov7.git
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-w6-pose.pt

Cloning into 'yolov7'...
remote: Enumerating objects: 1197, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 1197 (delta 2), reused 3 (delta 1), pack-reused 1191
Receiving objects: 100% (1197/1197), 74.24 MiB | 24.73 MiB/s, done.
Resolving deltas: 100% (518/518), done.


In [12]:
import torch
from torchvision import transforms
import sys
sys.path.append('/content/yolov7')

from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts

import cv2
import csv
import os
import math
import datetime
import numpy as np

In [13]:
input = 'file'

if input == 'file':
  # SET AN INPUT FILE in the folder cropped_videos
  input_file = 'output_video_-M5SITXMA2Y.mp4'
  url = f"https://github.com/Strojove-uceni/23206-final-pose-estimation-for-swing-improvement/raw/main/cropped_videos/{input_file}"
  !wget {url}

  csv_file_name = f'{os.path.splitext(input_file)[0]}.csv'
  output_video_name = f'{os.path.splitext(input_file)[0]}_output.mp4'

elif input == 'folder':
  !git clone https://github.com/Strojove-uceni/23206-final-pose-estimation-for-swing-improvement
  path = '/content/23206-final-pose-estimation-for-swing-improvement/cropped_videos'
  sys.path.append(path)
  input_folder = path #download the folder cropped_videos
  output_folder_videos = 'result_videos'
  output_folder_csv = 'result_statistics'
  # Create output folders if they don't exist
  os.makedirs(output_folder_videos, exist_ok=True)
  os.makedirs(output_folder_csv, exist_ok=True)

else:
  print("You have just two options: 'file' or 'folder'")

--2023-12-12 09:27:30--  https://github.com/Strojove-uceni/23206-final-pose-estimation-for-swing-improvement/raw/main/cropped_videos/output_video_-M5SITXMA2Y.mp4
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Strojove-uceni/23206-final-pose-estimation-for-swing-improvement/main/cropped_videos/output_video_-M5SITXMA2Y.mp4 [following]
--2023-12-12 09:27:31--  https://raw.githubusercontent.com/Strojove-uceni/23206-final-pose-estimation-for-swing-improvement/main/cropped_videos/output_video_-M5SITXMA2Y.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 254614 (249K) [application/octet-stream]
Savi

In [16]:
# method to load the model from the downloaded weights
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def calculate_angle(a_x, a_y, b_x, b_y, c_x, c_y):
    # Calculate the angle between three points in degrees
    radians = math.atan2(c_y - b_y, c_x - b_x) - math.atan2(a_y - b_y, a_x - b_x)
    angle = math.degrees(radians)
    angle = abs(angle)
    if angle > 180:
        angle = 360 - angle
    return angle

def calculate_angle2(x1, y1, x2, y2, axis='x', orientation='right'):
    if (math.sqrt((x2 - x1)**2 + (y2 - y1)**2) * x1) != 0:
        if axis == 'x':
            theta = math.acos((x2 - x1) * (-x1) / (math.sqrt((x2 - x1)**2 + (y2 - y1)**2) * x1))
        elif axis == 'y':
            theta = math.acos( (y2 -y1)*(-y1) / (math.sqrt((x2 - x1)**2 + (y2 - y1)**2 ) * y1))
        else:
            raise ValueError("Invalid axis, use 'x' or 'y'")

        if orientation == 'right':
            angle = int(180/math.pi) * theta
        elif orientation == 'left':
            angle = 180 - int(180/math.pi) * theta
        else:
            raise ValueError("Invalid orientation, use 'left' or 'right'")

    else:
        return 0  # Handle the case where x1 is zero to avoid division by zero

    return angle

def middle_point(a_x, a_y, b_x, b_y):
    midpoint_x = (a_x + b_x) / 2
    midpoint_y = (a_y + b_y) / 2
    return midpoint_x, midpoint_y


def load_model():
    model = torch.load('yolov7-w6-pose.pt', map_location=device)['model']
    # Put in inference mode
    model.float().eval()

    if torch.cuda.is_available():
        # half() turns predictions into float16 tensors
        # which significantly lowers inference time
        model.half().to(device)
    return model


model = load_model()


# method for running inference
def run_inference(image):
    # Resize and pad image
    image = letterbox(image, 960, stride=64, auto=True)[0]  # shape: (567, 960, 3)
    # Apply transforms
    image = transforms.ToTensor()(image)  # torch.Size([3, 567, 960])
    if torch.cuda.is_available():
        image = image.half().to(device)
    # Turn image into batch
    image = image.unsqueeze(0)  # torch.Size([1, 3, 567, 960])
    with torch.no_grad():
        output, _ = model(image)
    return output, image


def save_keypoints(kpts, steps):
    num_kpts = len(kpts) // steps
    coords = []
    for kid in range(num_kpts):
        x_coord, y_coord = kpts[steps * kid], kpts[steps * kid + 1]
        if not (x_coord % 640 == 0 or y_coord % 640 == 0):
            if steps == 3:
                conf = kpts[steps * kid + 2]
                if conf < 0.5:
                    coords.append(0)
                    coords.append(0)
                    continue
        coords.append(x_coord)
        coords.append(y_coord)
    return coords


# return the predictions of the model and plot the predicted skeletons over the image itself
def draw_keypoints(output, image):
    output = non_max_suppression_kpt(output,
                                      0.25,  # Confidence Threshold
                                      0.65,  # IoU Threshold
                                      nc=model.yaml['nc'],  # Number of Classes
                                      nkpt=model.yaml['nkpt'],  # Number of Keypoints
                                      kpt_label=True)
    with torch.no_grad():
        output = output_to_keypoint(output)
    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)

    all_coords = []
    for idx in range(output.shape[0]):
        #plot_skeleton_kpts(nimg, output[idx, 7:].T, 3)
        coords = save_keypoints(output[idx, 7:].T, 3)
        all_coords.append(coords)
    return nimg, all_coords


def write_csv_file(video_name, coordinates):
    if input == 'file':
      csv_filename = video_name
    else:
      csv_folder = 'result_statistics'
      os.makedirs(csv_folder, exist_ok=True)
      csv_filename = os.path.join(csv_folder, f'{video_name}.csv')

    with open(csv_filename, mode='a', newline='') as file:
        writer = csv.writer(file)

        # Check if the file is empty (i.e., first frame) and write header accordingly
        if os.path.getsize(csv_filename) == 0:
            writer.writerow(["video_timestamp", "shoulders_inclination", "hips_inclination",
                             "knee_angle", "pelvis_angle", "arm_angle",
                             "right_shoulder X", "right_shoulder Y",
                             "left_shoulder X", "left_shoulder Y",
                             "left_elbow X", "left_elbow Y",
                             "right_wrist X", "right_wrist Y",
                             "left_wrist X", "left_wrist Y",
                             "nose X", "nose Y",
                             "right_hip X", "right_hip Y",
                             "left_hip X", "left_hip Y",
                             "right_knee X", "right_knee Y",
                             "right_ankle X", "right_ankle Y",
                             "left_ankle X", "left_ankle Y",
                             "midpoint X", "midpoint Y"])

        writer.writerow(coordinates)


# real-time video setting
def pose_estimation(input_file, csv_file_name, output_video_name):
    output_folder = 'result_videos'
    os.makedirs(output_folder, exist_ok=True)

    cap = cv2.VideoCapture(input_file)
    # VideoWriter for saving the video
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(output_video_name, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))
    frame_number = 0
    while cap.isOpened():
        (ret, frame) = cap.read()
        if ret == True:
            frame_number += 1

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            output, frame = run_inference(frame)
            frame, all_coords = draw_keypoints(output, frame)

            a = 0
            for person in all_coords:
                # Get coordinates of specific landmarks
                left_shoulder_x = person[10]
                left_shoulder_y = person[11]
                right_shoulder_x = person[12]
                right_shoulder_y = person[13]
                left_hip_x = person[22]
                left_hip_y = person[23]
                right_hip_x = person[24]
                right_hip_y = person[25]
                left_wrist_x = person[18]
                left_wrist_y = person[19]
                right_wrist_x = person[20]
                right_wrist_y = person[21]
                nose_x = person[0]
                nose_y = person[1]
                left_knee_x = person[26]
                left_knee_y = person[27]
                right_knee_x = person[28]
                right_knee_y = person[29]
                left_ankle_x = person[30]
                left_ankle_y = person[31]
                right_ankle_x = person[32]
                right_ankle_y = person[33]
                left_elbow_x = person[14]
                left_elbow_y = person[15]

                # Calculate angles
                knee_angle = calculate_angle(left_hip_x, left_hip_y, left_knee_x, left_knee_y, left_ankle_x, left_ankle_y)
                pelvis_angle = calculate_angle(left_ankle_x, left_ankle_y, left_hip_x, left_hip_y, right_shoulder_x, right_shoulder_y)
                arm_angle = calculate_angle(left_wrist_x, left_wrist_y, left_elbow_x, left_elbow_y, left_shoulder_x, left_shoulder_y)
                shoulders_inclination = calculate_angle2(right_shoulder_x, right_shoulder_y,
                                                         left_shoulder_x, left_shoulder_y, 'x', 'left')
                hips_inclination = calculate_angle2(left_hip_x, left_hip_y, right_hip_x, right_hip_y, 'x')
                midpoint_x, midpoint_y = middle_point(right_ankle_x, right_ankle_y, left_ankle_x, left_ankle_y)
                # Get the timestamp
                fps = cap.get(cv2.CAP_PROP_FPS)
                video_timestamp = round(frame_number / fps)
                video_timestamp = str(datetime.timedelta(seconds=video_timestamp))


                # Display points
                cv2.circle(frame, (int(right_shoulder_x), int(right_shoulder_y)), 6, (0, 255, 0), -1)
                cv2.circle(frame, (int(left_shoulder_x), int(left_shoulder_y)), 6, (0, 255, 0), -1)
                cv2.circle(frame, (int(right_hip_x), int(right_hip_y)), 6, (255, 255, 0), -1)
                cv2.circle(frame, (int(left_hip_x), int(left_hip_y)), 6, (0, 150, 255), -1)
                cv2.circle(frame, (int(right_knee_x), int(right_knee_y)), 6, (255, 0, 255), -1)
                cv2.circle(frame, (int(left_knee_x), int(left_knee_y)), 6, (255, 0, 255), -1)
                cv2.circle(frame, (int(left_ankle_x), int(left_ankle_y)), 6, (255, 0, 0), -1)
                cv2.circle(frame, (int(left_wrist_x), int(left_wrist_y)), 6, (0, 255, 255), -1)
                cv2.circle(frame, (int(nose_x), int(nose_y)), 6, (0, 0, 255), -1)
                cv2.circle(frame, (int(left_elbow_x), int(left_elbow_y)), 6, (128, 0, 128), -1)
                cv2.circle(frame, (int(right_ankle_x), int(right_ankle_y)), 6, (255, 0, 0), -1)
                cv2.circle(frame, (int(midpoint_x), int(midpoint_y)), 6, (255, 255, 255), -1)

                # Display angle and lines on the image
                cv2.line(frame, (int(left_ankle_x), int(left_ankle_y)), (int(left_ankle_x), int(left_ankle_y)- 200), (255, 0, 0), 2)
                cv2.line(frame, (int(right_ankle_x), int(right_ankle_y)), (int(right_ankle_x), int(right_ankle_y) - 200), (255, 0, 0), 2)

                cv2.line(frame, (int(right_shoulder_x), int(right_shoulder_y)), (int(right_shoulder_x) + 100, int(right_shoulder_y)), (0, 255, 0), 2)
                cv2.line(frame, (int(left_shoulder_x), int(left_shoulder_y)), (int(right_shoulder_x), int(right_shoulder_y)), (0, 255, 0), 2)
                cv2.line(frame, (int(left_hip_x), int(left_hip_y)), (int(left_hip_x) - 100, int(left_hip_y) ), (255, 255, 0), 2)
                cv2.line(frame, (int(left_hip_x), int(left_hip_y)), (int(right_hip_x), int(right_hip_y)), (255, 255, 0), 2)

                cv2.line(frame, (int(left_hip_x), int(left_hip_y)), (int(left_knee_x), int(left_knee_y)), (255, 0, 255), 2)
                cv2.line(frame, (int(left_knee_x), int(left_knee_y)), (int(left_ankle_x), int(left_ankle_y)), (255, 0, 255), 2)

                cv2.line(frame, (int(left_hip_x), int(left_hip_y)), (int(left_ankle_x), int(left_ankle_y)), (0, 150, 255), 2)
                cv2.line(frame, (int(left_hip_x), int(left_hip_y)), (int(right_shoulder_x), int(right_shoulder_y)), (0, 150, 255), 2)

                cv2.line(frame, (int(left_shoulder_x), int(left_shoulder_y)), (int(left_elbow_x), int(left_elbow_y)), (128, 0, 128), 2)
                cv2.line(frame, (int(left_elbow_x), int(left_elbow_y)), (int(left_wrist_x), int(left_wrist_y)), (128, 0, 128), 2)
                cv2.line(frame, (int(midpoint_x), int(midpoint_y)), (int(midpoint_x), int(midpoint_y) - 200), (255, 255, 255), 2)

                # Write coordinates to CSV file
                if input == 'file':
                  video_name = csv_file_name
                else:
                  if a == 0:
                      video_name = filename.replace('.mp4', '')
                  else:
                      video_name = filename.replace('.mp4', f'_{a+1}')

                write_csv_file(video_name, [video_timestamp, shoulders_inclination, hips_inclination,
                                            knee_angle, pelvis_angle, arm_angle,
                                            right_shoulder_x, right_shoulder_y,
                                            left_shoulder_x, left_shoulder_y,
                                            left_elbow_x, left_elbow_y,
                                            right_wrist_x, right_wrist_y,
                                            left_wrist_x, left_wrist_y,
                                            nose_x, nose_y,
                                            right_hip_x, right_hip_y,
                                            left_hip_x, left_hip_y,
                                            right_knee_x, right_knee_y,
                                            right_ankle_x, right_ankle_y,
                                            left_ankle_x, left_ankle_y,
                                            midpoint_x, midpoint_y])

                a = a+1

            if a == 1:
                cv2.putText(frame, f'Shoulders inclination: {shoulders_inclination:.2f}', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                cv2.putText(frame, f'Hips inclination: {hips_inclination:.2f}', (10, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
                cv2.putText(frame, f'Knee Angle: {knee_angle:.2f}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2)
                cv2.putText(frame, f'Pelvis Angle: {pelvis_angle:.2f}', (10, 95), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 150, 255), 2)
                cv2.putText(frame, f'Arm Angle: {arm_angle:.2f}', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (128, 0, 128), 2)


            frame = cv2.resize(frame, (int(cap.get(3)), int(cap.get(4))))
            out.write(frame)
        else:
            break
        if cv2.waitKey(5) & 0xFF == 27:
            break

    cap.release()
    out.release()

cv2.destroyAllWindows()

In [17]:
if input == 'file':
  pose_estimation(input_file, csv_file_name, output_video_name)
else:
  for filename in os.listdir(input_folder):
    if filename.endswith(".mp4"):
        # Set output file names
        csv_file_name = f'{os.path.splitext(filename)[0]}.csv'
        output_video_name = f'{os.path.splitext(filename)[0]}_output.mp4'
        csv_file_path = os.path.join(output_folder_csv, csv_file_name)
        output_video_path = os.path.join(output_folder_videos, output_video_name)
        file_path = os.path.join(input_folder, filename)
        pose_estimation(file_path, csv_file_path, output_video_path)